In [1]:
from sklearn.model_selection import KFold
from scipy.io import loadmat
import numpy as np
from models import *
from time import time 

# set random seed
np.random.seed(42)

In [2]:
dataset = loadmat('coil20.mat')
label = np.array([dataset['Y'][i][0] - 1 for i in range(len(dataset['Y']))])
data = dataset['X']
n_class = len(np.unique(label))

# train-test-split
# X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2)
lams = [2**i for i in range(-6, 13, 2)] # regularization parameter, lambda

In [3]:
def cross_val_acc(data, label, n_class, model_class, lam=None, n_layer=1, activation='sigmoid'):
    n_node = 100 # num of nodes in hidden layer
    w_range = [-1, 1] # range of random weights
    b_range = [0, 1] # range of random biases
    val_acc = []
    train_time = []
    test_time = []

    kf = KFold(n_splits=10, shuffle=True)
    # print('Lambda: ', lam)
    for i, kf_values in enumerate(kf.split(data, label)):
    #     print(f'train: {train_index}, val: {val_index}')
        # print('Validation: {}'.format(i + 1))
        train_index, val_index = kf_values
        X_val_train, X_val_test = data[train_index], data[val_index]
        y_val_train, y_val_test = label[train_index], label[val_index]
        if model_class in [RVFL, DeepRVFL, EnsembleDeepRVFL]:
            model = model_class(n_node, lam, w_range, b_range, n_layer, activation=activation)
        elif model_class in [BRVFL, BDeepRVFL, BEnsembleDeepRVFL]:
            model = model_class(n_node, w_range, b_range, n_layer, tol=10**(-7), activation=activation)
        t = time()
        model.train(X_val_train, y_val_train, n_class)
        train_t = time()
        acc = model.eval(X_val_test, y_val_test)
        test_t = time()
        train_time.append(train_t - t)
        test_time.append(test_t - train_t)

        # print(f'Validation accuracy: {acc}')
        val_acc.append(acc)

    acc = [np.mean(val_acc), np.std(val_acc)]
    t = [np.mean(train_time), np.mean(test_time)]
    # print(f'Model accuracy: {mean_acc}\n')
    return model, acc, t

In [4]:
# RVFL
acc = []
t = []
for lam in lams:
    _, model_accuracy, duration = cross_val_acc(data, label, n_class, RVFL, lam)
    acc.append(model_accuracy)
    t.append(duration)

max_index = np.argmax(acc, axis=0)[0]
opt_lam = lams[max_index]
print('Accuracy: ', acc[max_index][0], u"\u00B1", acc[max_index][1])
print('Lambda: ', opt_lam)
print('Train time: ', t[max_index][0])
print('Test time: ', t[max_index][1])

Accuracy:  0.9673611111111111 ± 0.008246070893776327
Lambda:  64
Train time:  0.24708056449890137
Test time:  0.0026417970657348633


In [23]:
# dRVFL
acc = []
t = []
for lam in lams:
    _, model_accuracy, duration = cross_val_acc(data, label, n_class, DeepRVFL, lam, n_layer=5)
    acc.append(model_accuracy)
    t.append(duration)

max_index = np.argmax(acc, axis=0)[0]
opt_lam = lams[max_index]
print('Accuracy: ', acc[max_index][0], u"\u00B1", acc[max_index][1])
print('Lambda: ', opt_lam)
print('Train time: ', t[max_index][0])
print('Test time: ', t[max_index][1])

Accuracy:  0.9715277777777779 ± 0.011805555555555548
Lambda:  16
Train time:  0.523043179512024
Test time:  0.007701730728149414


In [24]:
# edRVFL
acc = []
t = []
for lam in lams:
    _, model_accuracy, duration = cross_val_acc(data, label, n_class, EnsembleDeepRVFL, lam, n_layer=5)
    acc.append(model_accuracy)
    t.append(duration)

max_index = np.argmax(acc, axis=0)[0]
opt_lam = lams[max_index]
print('Accuracy: ', acc[max_index][0], u"\u00B1", acc[max_index][1])
print('Lambda: ', opt_lam)
print('Train time: ', t[max_index][0])
print('Test time: ', t[max_index][1])

/home/yc0009/projects/python/FYP/models.py:33: RuntimeWarning: overflow encountered in power
  return 1 / (1 + np.e ** (-x))


Accuracy:  0.9680555555555556 ± 0.019194826334840626
Lambda:  64
Train time:  1.3181850910186768
Test time:  0.01643509864807129


In [4]:
# BRVFL
model, acc, t = cross_val_acc(data, label, n_class, BRVFL)

print('Accuracy: ', acc[0], u"\u00B1", acc[1])
print('Train time: ', t[0])
print('Test time: ', t[1])
print('Hyperparameters: ')
print('Precision: ', model.prec)
print('Variance: ', model.var)


Convergence after  27  iterations



Convergence after  26  iterations



Convergence after  27  iterations



Convergence after  26  iterations



Convergence after  26  iterations



Convergence after  26  iterations



Convergence after  26  iterations



Convergence after  27  iterations



Convergence after  26  iterations



Convergence after  26  iterations
Accuracy:  0.9395833333333334 ± 0.022405924729030104
Train time:  83.17574417591095
Test time:  0.003948736190795899
Hyperparameters: 
Precision:  66.35566003205462
Variance:  0.002856025797307477


In [6]:
# BdRVFL
model, acc, t = cross_val_acc(data, label, n_class, BDeepRVFL, n_layer=5)

print('Accuracy: ', acc[0], u"\u00B1", acc[1])
print('Train time: ', t[0])
print('Test time: ', t[1])
print('Hyperparameters: ')
print('Precision: ', model.prec)
print('Variance: ', model.var)

In [5]:
# BedRVFL
model, acc, t = cross_val_acc(data, label, n_class, BEnsembleDeepRVFL, n_layer=5)

print('Accuracy: ', acc[0], u"\u00B1", acc[1])
print('Train time: ', t[0])
print('Test time: ', t[1])
print('Hyperparameters: ')
print('Precision: ', model.prec)
print('Variance: ', model.var)

/home/yc0009/projects/python/FYP/models.py:33: RuntimeWarning: overflow encountered in power
  return 1 / (1 + np.e ** (-x))



Convergence after  27  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  22  iterations


/home/yc0009/projects/python/FYP/models.py:33: RuntimeWarning: overflow encountered in power
  return 1 / (1 + np.e ** (-x))



Convergence after  27  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  23  iterations



Convergence after  26  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  22  iterations


...


Convergence after  27  iterations
Convergence after  23  iterations
Convergence after  23  iterations
Convergence after  23  iterations
Convergence after  23  iterations



Convergence after  28  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  23  iterations
Convergence after  23  iterations



Convergence after  26  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  23  iterations
Convergence after  22  iterations



Convergence after  28  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  23  iterations
Convergence after  23  iterations



Convergence after  27  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  22  iterations
Convergence after  23  iterations
Accuracy:  0.9625000000000001 ± 0.008893228107545626
Train time:  139.65744345188142
Test time:  0.055353331565856936
Hyperparameters: 
Precision:  [73.98087943213655, 535.1351450826894, 581.4471537151932, 531.5625212043403, 527.7816718313668]
Variance:  [0.0038836090231726657, 0.18563509847970536, 0.22728726246125183, 0.18239806647494197, 0.17811290174797648]
